In [1]:
MOCK_BOOKS = [
    {
        "title": "Murder on the Orient Express",
        "price": 10.99,
        "author": "Agatha Christie",
    },
    {"title": "And Then There Were None", "price": 8.99, "author": "Agatha Christie"},
    {"title": "The ABC Murders", "price": 12.00, "author": "Agatha Christie"},
    {"title": "Death on the Nile", "price": 15.49, "author": "Agatha Christie"},
    {
        "title": "The Murder of Roger Ackroyd",
        "price": 9.99,
        "author": "Agatha Christie",
    },
]

In [74]:
def search_books_function(author: str, max_price: float):
    """
    Return a list of books for the given author cheaper than max_price.
    For the mock: return 5 books with random prices and filter them.
    """
    result = []

    for book in MOCK_BOOKS:
        if book["author"].lower() == author.lower() and book["price"] <= max_price:
            print("Auther exsists ")
            result.append(book)

    cheapest = sorted(result, key=lambda x:x['price'])
    return cheapest

In [62]:
search_books_function("Agatha Christie", 9)

Auther exsists 
[{'title': 'And Then There Were None', 'price': 8.99, 'author': 'Agatha Christie'}]


[{'title': 'And Then There Were None',
  'price': 8.99,
  'author': 'Agatha Christie'}]

In [63]:
search_books = {
    "name": "search_books_function",
    "description": "This is a simple function to look up the book by auther name and price",
    "input_schema": {
        "type": "object",
        "properties": {
            "operation": {
                "author": "string",
                "description": "The name of the book auther",
            },
            "price": {"type": "number", "description": "The price of the book"},
        },
        "required": ["author", "price"],
    },
}

In [89]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

# Fixed to use correct tool usage pattern expected by Anthropic
response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[
        {
            "role": "user",
            "content": "Are there any books for auther Agatha Christie under $10?",
        }
    ],
    max_tokens=400,
    tool_choice={"type": "auto"},
    system="You have access to tools, but only use them when necessary.  If a tool is not required, respond as normal",
    tools=[search_books],
)

response

Message(id='msg_01BBziUA8XoGBa5G8QikQ3nN', content=[ToolUseBlock(id='toolu_01VfJcZnUtK7XzpUFpXAKcCF', input={'author': 'Agatha Christie', 'price': 10}, name='search_books_function', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=412, output_tokens=75, server_tool_use=None, service_tier='standard'))

In [90]:
response.content[0].text

AttributeError: 'ToolUseBlock' object has no attribute 'text'

In [91]:
if response.stop_reason == "tool_use":
    tool = response.content[-1]
    print(tool)
    input = response.content[0].input
    auther = str(input["author"])
    price = input["price"]
    result = search_books_function(auther, price)
    print(result)
    
    final_response = {
        "role":"user",
        "content": {
            "type": "tool_result",
            "tool_use_id": tool.id,
            "content": str(result),
        }
    }
    
    print(final_response)
else:
    print(response.content[0].text)

ToolUseBlock(id='toolu_01VfJcZnUtK7XzpUFpXAKcCF', input={'author': 'Agatha Christie', 'price': 10}, name='search_books_function', type='tool_use')
Auther exsists 
Auther exsists 
[{'title': 'And Then There Were None', 'price': 8.99, 'author': 'Agatha Christie'}, {'title': 'The Murder of Roger Ackroyd', 'price': 9.99, 'author': 'Agatha Christie'}]
{'role': 'user', 'content': {'type': 'tool_result', 'tool_use_id': 'toolu_01VfJcZnUtK7XzpUFpXAKcCF', 'content': "[{'title': 'And Then There Were None', 'price': 8.99, 'author': 'Agatha Christie'}, {'title': 'The Murder of Roger Ackroyd', 'price': 9.99, 'author': 'Agatha Christie'}]"}}
